## Generalized World Coordinate System (GWCS)
#### Nadia Dencheva, Brett Graham, Perry Greenfield
#### AAS 245

### Why use GWCS for JWST and Roman?

- Why not FITS WCS?
  - FITS WCS has been very successful for regulary sampled images
  - Doesn't handle distortions
  - Handles only regular spectral data
  - Does not handle discontiguous data like JWST's IFUs and NIRSpec's MOS
  - Has all the deficiencies of the FITS format

### GWCS Data Model

- A GWCS pipeline is a list of steps
- Each step has a coordinate frame and a transform from this frame to the next one
- The transform of the last step is `None` and indicates the end of the pipeline

### Implementation

- [astropy.modeling](https://docs.astropy.org/en/stable/modeling/index.html) to implement transforms
- [astropy.coordinates](https://docs.astropy.org/en/stable/coordinates/index.html) for coordinate frames
- [astropy.units](https://docs.astropy.org/en/stable/units/index.html)
- implements the [Shared WCS API](https://docs.astropy.org/en/stable/wcs/wcsapi.html)
- serialized to file using ASDF

### Basic features

JWST products are serialized to FITS, with the WCS written to a special non-standard FITS extension with `EXTNAME=ASDF`, known as ASDF-in-FITS.

Roman will use ASDF as a native serialization format. While Roman files can be opened with asdf, we have developed data models for all products which provide additional convinience tools. What follows applies to both Roman and JWST, any differences are noted.

In [ ]:
# Common imports
from matplotlib import pyplot as plt
%matplotlib inline

import numpy as np

from gwcs import wcs
from gwcs import coordinate_frames as cf

In [ ]:
# import Roman datamodels
from roman_datamodels import datamodels as rdm

In [ ]:
# import JWST datamodels
from stdatamodels.jwst import datamodels 

In [ ]:
cal = rdm.open('../asdf/data/roman.asdf')

In [ ]:
cal.info()

In [ ]:
cal.search('wcs')

Searching for key `wcs` in the Roman calibration file gives several results.

These are mostly common with the JWST cal files as well.

- `meta.cal_step.assign_wcs`: a record that the assign_wcs step has been run on this file
- `meta.wcs`: the GWCS object
- `meta.wcs.fit_results`: a record of the tweakreg fitting parameters to GAIA (Roman specific)
- `meta.wcsinfo`: pointing parameters



In [ ]:
w = cal.meta.wcs

print(w)

### Shared vs Legacy API

The Shared WCS API consists of methods commmon to other WCS libraries, currently astropy FITS WCS and GWCS. It is useful if writing general applications which should work with FITS and ASDF files. 

##### Shared API

In [ ]:
ra, dec = w.pixel_to_world_values(100, 200)

print(f"World values (numerical): {ra, dec}")

x1, y1 = w.world_to_pixel_values(ra, dec)

print(f"Roundtripping values, world -> detector numerical values {x1, y1}")

In [ ]:
sky = w.pixel_to_world(100, 200)
print(f" World Sky object {sky}")
print(f"Type of sky is {type(sky)}")

In [ ]:
x1, y1 = w.world_to_pixel(sky)

print(f"Roundtripping values, world -> detector SkyCoord object {x1, y1}")

Using high level Python objects, like SkyCoord, allows all the functionality of astropy.coordinates to be used on the result of GWCS evaluation.

In [ ]:
sky.galactic

In [ ]:
sky.fk5

##### Legacy methods

These work only with numerical values and allow a little bit more flexibility.

To evaluate the forward transform, the GWCS object can be called like a function. To evaluate the backward transform use `invert`.

In [ ]:
ra, dec = w(100, 200)
print(f"World values: {ra, dec}")

x1, y1 = w.invert(ra, dec)

print(f"Roundtripping values, world -> detector{x1, y1}")

#### Performance hint
If the WCS object is called repeatedly and is not changing between calls, it is more performant to use 

`wcs.forward_transform` and `wcs.backward_transform`

The reason is that the transforms are computed dynamically every time `wcs.__call__` and `wcs.invert` are called and there's some overhead in generating the transform. The performance improvement depends on the WCS and how many transforms it is using.

In [ ]:
w.backward_transform(*w.forward_transform(100, 200))

##### The bounding box

A GWCS object has an attribute `bounding_box` which is an instance of `astropy.modeling.ModelBoundingBox` or `astropy.modeling.CompoundBoundingBox`. It represents the valid range of inputs. Results for inputs outside the bounding_box are set to np.NaN.

A GWCS object has a method `footprint` which returns a numerical value of the evaluation of the bounding_box. Inverse transformations take into account the footprint and th einput bounding_box to determine valid values. These can be disabled by passing `with_bounding_box=False` to the `wcs.__call__` or `wcs.invert`. 

The Shared API does not accept this keyword. To force the transforms to ignore the `bounding_box` when using the shared API, set the `wcs.bounding_box=None`.

##### Evaluating a grid of valid inputs

`gwcs.wcstools.grid_from_bounding_box` returns a grid of input values within the `bounding_box`, their number corresponding to the number of input axes.

#### Exercise 1:

Using the file above

- Open the file and read the WCS
- Display the bounding_box
- Generate the grid of inputs
  Extra bonus
  - generate a grid using the edge of the pixels instead of the centers (hint: use the `center` parameter)
  - generate a grid with sampling every 5 pixels (hint: use the `step` parameter)
- Use the Shared and Legacy APIs to generate world coordinates
- Evaluate the WCS on x=4088 and y=4088
- Compute the footprint of the image on sky

### Transforms

GWCS's transforms are based on astropy.modeling. The flexibility of GWCS comes mostly from the various ways the models can be combined:

- using arithmetic operators
- in series
- in parallel
- inputs can be fixed, reordered, dropped or added

What follows is a brief introduction to astropy.modeling.

In [ ]:
jcal = datamodels.open('../asdf/data/jwst.fits')

In [ ]:
jw = jcal.meta.wcs
print(jw)

In [ ]:
print(jw.forward_transform)

### Exercise 2:

- Use the prescription above to generate a compound model with different parameters.
- Using this new transform generate a GWCS object. The coordinate frames can be accessed by `wcs.input_frame` and `wcs.output_frame`.

### WCS methods

We will use a Nirspec Level 2 (cal) file to demonstrate other GWCS features. This product is a `MultiSlitModel` from a MOS observation. 

In [ ]:
jnrs = datamodels.open('../asdf/data/jw04291004001_13101_00001_nrs2_cal.fits')
print(f"The data product is of type {type(jnrs)}")

print(f"The number of extracted slits from the MSA is {len(jnrs.slits)}")

In [ ]:
s50 = jnrs.slits[50]
w0 = s50.meta.wcs

In [ ]:
plt.imshow(s50.data, origin='lower', vmin=.1, vmax=.3, aspect='auto')

plt.gray()

In [ ]:
print(w0)

This WCS has 9 coordinate frames. We will demonstrate

- retrieving a transform between two intermediate frames
- modifying a transform
- setting a transform
- insert a frame and a transform

In [ ]:
# Choose coordinates around the middle of the bounding box and find the corresponding coordinates in the virtual slit
print(f"bounding_box is {w0.bounding_box}")

In [ ]:
det2slit = w0.get_transform('detector', 'slit_frame')

In [ ]:
x_slit, y_slit, lam = det2slit(1000, 10.2)

print(f"Slit coordinates: {x_slit, y_slit, lam}")

The coordinate frame "v2v3" is a spherical system associated with the telescope. 
The transform from "v2v3" to "v2v3vacorr" correct for the velocity aberration. Let's remove the aberration correction and the frame associated with it.

In [ ]:
# Let assign a name to the transform from slit_frame to msa_frame

slit2msa = w0.get_transform('slit_frame', 'msa_frame')
print(slit2msa.name)

In [ ]:
slit2msa.name = "slit2msa"
w0.set_transform('slit_frame', 'msa_frame', slit2msa)
print(w0)

In [ ]:
import copy

w0copy = copy.deepcopy(w0)


In [ ]:
from astropy.modeling.models import Shift, Identity

# create a new frame associated with shifted RA, DEC coordinates. For convenience we will copy the output "world" frame.
ra_dec_shift = copy.deepcopy(w0.output_frame)
ra_dec_shift.name = 'ra_dec_shift'

# Create a transform which shifts RA, DEC by small a few hundreth of an arcsec and keeps the wavelength the same
m = Shift(1) & Shift(1) & Identity(1)
m.name = "offset_ra_dec"

In [ ]:
w0copy.insert_frame(ra_dec_shift, m, 'world')

In [ ]:
print(w0)


In [ ]:
print(w0copy)

In [ ]:
print(w0copy.get_transform('ra_dec_shift', 'world'))

In [ ]:
print(f"old RA, DEC: {w0(1000, 10.5)}")
print(f"new RA, DEC: {w0copy(1000, 10.5)}")